In [74]:
import nltk
from nltk.stem import WordNetLemmatizer

import re
from bs4 import BeautifulSoup
import spacy
import numpy as np

In [77]:
lemmatizer = WordNetLemmatizer()
print (lemmatizer.lemmatize('working', pos='v'))

dog


In [80]:
def load_data(FILE_Path):
    otuput_sent = []
    lemmatizer = WordNetLemmatizer()
    with open(FILE_Path, "r") as f:
        train_tmp = f.read().split("\n\n")[:-1]
        for sample in train_tmp:
            sentence = re.split('\t|\n', sample)[1].replace('\"',"").replace(".","").replace(",","")\
            .replace("?","").replace("!","").replace(")","").replace("(","").replace("\'s","")\
            .replace("\'ve","").replace("\'t","").replace("\'re","").replace("\'d","").replace("\'ll","")
            answer = re.split('\t|\n', sample)[2]
            soup = BeautifulSoup(sentence,"lxml")

            e1 = str(soup.find('e1'))[4:-5]
            e2 = str(soup.find('e2'))[4:-5]
            word_list = sentence.split(" ")
            for word in word_list:
                if "</e1>" in word:
                    a = word
                if "</e2>" in word:
                    b = word
            p1 = word_list.index(a)
            p2 = word_list.index(b)
            sentence = sentence.replace("<e1>","").replace("</e1>","").replace("<e2>","").\
            replace("</e2>","")
            sentence = [lemmatizer.lemmatize(word, pos='v') for word in sentence.split(" ")]
            sentence = " ".join(sentence)
            otuput_sent.append((sentence,e1,e2,p1,p2,answer))
    return otuput_sent

In [81]:
train_instance = load_data("./data/TRAIN_FILE.txt")
test_instance = load_data("./data/TEST_FILE_FULL.txt")

In [82]:
train_instance

[('The system as describe above have its greatest application in an array configuration of antenna elements',
  'configuration',
  'elements',
  12,
  15,
  'Component-Whole(e2,e1)'),
 ('The child be carefully wrap and bind into the cradle by mean of a cord',
  'child',
  'cradle',
  1,
  9,
  'Other'),
 ('The author of a keygen use a disassembler to look at the raw assembly code',
  'author',
  'disassembler',
  1,
  7,
  'Instrument-Agency(e2,e1)'),
 ('A misty ridge uprise from the surge', 'ridge', 'surge', 2, 6, 'Other'),
 ('The student association be the voice of the undergraduate student population of the State University of New York at Buffalo',
  'student',
  'association',
  1,
  2,
  'Member-Collection(e1,e2)'),
 ('This be the sprawl complex that be Peru largest producer of silver',
  'complex',
  'producer',
  4,
  9,
  'Other'),
 ('The current view be that the chronic inflammation in the distal part of the stomach cause by Helicobacter pylori infection result in an increase 

In [39]:
train_position = [(ele[3],ele[4]) for ele in train_instance]

In [40]:
# check p1 p2 order
for a, b in train_position:
    if a > b: 
        print(a,b)

In [43]:
len(train_instance)

8000

### depency parsing

In [45]:
nlp = spacy.load('en_core_web_sm')

In [49]:
train_text = [ele[0] for ele in train_instance]
test_test = [ele[0] for ele in test_instance]

In [50]:
demo_id = 62
demo_sentence = train_text[demo_id]
print(demo_sentence)

doc = nlp(demo_sentence)

for pos, token in enumerate(doc):
    if pos in train_position[demo_id]:
        ancestors = [i for i in token.ancestors]
        print(token.text, token.tag_, token.dep_,
              token.shape_, token.is_stop, token.ent_type,ancestors,
              "lefts",[word.text for word in token.lefts],
              "rights",[word.text for word in token.rights],
              token.n_lefts,  # 2
            token.n_rights  # 1
             )

A great wing of fluorite dragons the greatest concentration of the rather solitary dragons ever known joined the desperate battle with the blues and silvers against Chaos
wing NN nsubj xxxx False 0 [concentration, joined] lefts ['A', 'great'] rights ['of'] 2 1
dragons NNS pobj xxxx False 0 [of, wing, concentration, joined] lefts ['fluorite'] rights [] 1 0


In [65]:
train_mut_ancestors_list = []
for ele in train_instance:
    sentence = ele[0]
    doc = nlp(sentence)
    entity = ele[1:3]
    both_ancestors = []
    for pos, token in enumerate(doc):
        if pos in ele[3:5]:
            ancestors = [i.text for i in token.ancestors]
            both_ancestors.append(ancestors)
    a = 0
    b = 0
    if entity[0] in both_ancestors[1]:
        a = 1
    elif entity[1] in both_ancestors[0]:
        b = 1
    mut_ancestors = [a,b]
    train_mut_ancestors_list.append(mut_ancestors)
    
test_mut_ancestors_list = []
for ele in test_instance:
    sentence = ele[0]
    doc = nlp(sentence)
    entity = ele[1:3]
    both_ancestors = []
    for pos, token in enumerate(doc):
        if pos in ele[3:5]:
            ancestors = [i.text for i in token.ancestors]
            both_ancestors.append(ancestors)
    a = 0
    b = 0
    if entity[0] in both_ancestors[1]:
        a = 1
    elif entity[1] in both_ancestors[0]:
        b = 1
    mut_ancestors = [a,b]
    test_mut_ancestors_list.append(mut_ancestors)

In [73]:
np.save("./features/train_mut_ancestors_list.npy", train_mut_ancestors_list)
np.save("./features/test_mut_ancestors_list.npy", test_mut_ancestors_list)